Generate prompts.

Methods:
- pairwise comparison via logits (original PairS method)
- direct-scoring via logits
- activation harvesting

Models:
- Mistral 7B v0.1 Instruct
- Llama 2 7B Chat
- Llama 3 8B Chat

Datasets
- SummEval
- News Room

In [1]:
import sys; sys.path.append("../utils")
from constants import *
from prompts import *
from utils import get_comparisons

import pandas as pd
from tqdm import tqdm, trange
from IPython.display import clear_output


instruction_theirs = lambda inst, article, s1, s2, aspect: theirs_compare.format(
    INSTRUCTION=inst,
    ARTICLE=article,
    SUMMARY1=s1,
    SUMMARY2=s2,
    ASPECT=aspect
)
answer_theirs = lambda _1, _2: "Answer: "

instruction_mine = lambda _, article, s1, s2, aspect: mine_compare.format(
    ARTICLE=article,
    SUMMARY1=s1,
    SUMMARY2=s2,
    ASPECT=aspect
)
answer_mine = lambda aspect, choice: f"Between Choice 1 and Choice 2, the more {aspect} summary is Choice {choice}"

In [5]:
for dataset in ["summeval", "newsroom"]    
    for model in ["mistral", "llama2", "llama3"]:

        path = f"{gdrive_path}/data/{dataset}-processed.jsonl"
        summaries = pd.read_json(path, orient="records", lines=True)

        # pairwise comparisons
        path = f"{gdrive_path}/prompts/{model}/{dataset}"
        chat_template = chat_templates[model]
        for i in range(2):
            instruction = [instruction_theirs, instruction_mine][i]
            answer = [answer_theirs, answer_mine][i]
            name = ["theirs", "mine"][i]
            filename = f"{path}_{name}"
            args = [summaries, chat_template, instruction, answer]
            if name == "mine":
                for choice in [1, 2]:
                    comparisons = get_comparisons(*args, choice)
                    comparisons.to_json(f"{filename}_{choice}.jsonl", orient="records", lines=True)
            else:
                comparisons = get_comparisons(*args)
                comparisons.to_json(f"{filename}.jsonl", orient="records", lines=True)

        # direct-scoring
        for aspect in ["coherence", "consistency", "fluency", "relevance"]:
            prompts = []
            for _, row in summaries.iterrows():
                prompt = theirs_score.format(
                    INSTRUCTION=theirs_instructions[aspect],
                    ARTICLE=row["article"],
                    SUMMARY=row["summary"],
                    ASPECT=aspect_noun2adj[aspect]
                )
                prompt = chat_templates[model].format(
                    INSTRUCTION=prompt,
                    ANSWER="Score: "
                )
                prompts.append(prompt)
            summaries[f"prompt_{aspect}"] = prompts
        summaries.to_json(f"{gdrive_path}/prompts/{model}/{dataset}_score.jsonl", orient="records", lines=True)

        clear_output()